In [1]:
%load_ext autoreload
%autoreload 2
%pylab inline
import pandas as pd
import os

Populating the interactive namespace from numpy and matplotlib


In [2]:
import pprint
pp = pprint.PrettyPrinter(indent=4)

# Dataframes
command: disjoint speaker split

In [3]:
dataframe_dir = '/home/muncok/DL/dataset/SV_sets/dataframes/'
data_dir = '/home/muncok/DL/dataset/SV_sets/reddots_r2015q4_v1/wav/'
data_df = pd.read_pickle('/home/muncok/DL/dataset/SV_sets/dataframes/Reddots_Dataframe.pkl')

In [4]:
import honk_sv.train as hk
import honk_sv.model as mod

In [5]:
big_sents = data_df.sent.value_counts().index[:10]
big_sent_df = data_df[data_df.sent.isin(big_sents)]

In [19]:
# sentence duration
import librosa

audio_lengths = []
for idx, row in big_sent_df.iterrows():
    file_path = os.path.join(data_dir, row.spk, row.file)
    x, sr = librosa.load(file_path, sr=16000)
    audio_lengths.append(len(x)/16000)

### Dataset split

In [6]:
#TODO: correct splits
all_spks = data_df.spk.unique()
all_sents = data_df.sent.unique()
uttrs_counts = data_df.spk.value_counts()
sv_spks = list(uttrs_counts.index[-8:])
si_spks = list(uttrs_counts.index[:-8])

In [21]:
# only big sents
data_df = big_sent_df
all_spks = data_df.spk.unique()
all_sents = data_df.sent.unique()
uttrs_counts = data_df.spk.value_counts()
sv_spks = sorted(list(uttrs_counts.index[-20:]))
si_spks = list(uttrs_counts.index[:-20])

In [56]:
sv_spks

['f0001',
 'f0003',
 'f0010',
 'f0011',
 'f0013',
 'f0015',
 'm0004',
 'm0010',
 'm0025',
 'm0030',
 'm0035',
 'm0037',
 'm0045',
 'm0050',
 'm0056',
 'm0059',
 'm0062',
 'm0063',
 'm0064',
 'm0067']

# SI Train

In [23]:
import random 

si_df = data_df[data_df.spk.isin(si_spks)]

# random sampling
si_random_train = si_df.sample(frac=0.8)
si_random_test = si_df.drop(index=si_random_train.index)
si_random_val = si_random_test.sample(frac=0.5)
si_random_test = si_random_test.drop(index=si_random_val.index) 
print("[random] train:{}, val:{}, test:{}".format(len(si_random_train), len(si_random_val), len(si_random_test)))

[random] train:5232, val:654, test:654


In [24]:
import csv
import random

tags = ['train', 'val', 'test']
# choose a si split
sets = {'train':si_random_train, 'val':si_random_val, 'test':si_random_test}
manifest_dir = "manifests/reddots/"

for tag in tags:
    samples = []
    save_path = os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))
    with open(save_path, 'w') as f:
        for index, row in sets[tag].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = si_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        random.shuffle(samples)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))

manifests/reddots/si_reddots_train_manifest.csv was written
manifests/reddots/si_reddots_val_manifest.csv was written
manifests/reddots/si_reddots_test_manifest.csv was written


In [7]:
model = "res8"
dataset = "reddots"

global_config = dict(model=model, dataset=dataset,
                     no_cuda=False,  gpu_no=0,
                     n_epochs=30, batch_size=64,
                     lr=[0.01], schedule=[np.inf], dev_every=1, seed=0, use_nesterov=False,
                     cache_size=32768, momentum=0.9, weight_decay=0.00001,
                     num_workers=16, print_step=100,
                     splice_length = 20
                     )

builder = hk.ConfigBuilder(
                mod.find_config(model),
                mod.SpeechDataset.default_config(dataset),
                global_config)
parser = builder.build_argparse()
si_config = builder.config_from_argparse(parser)
si_config['model_class'] = mod.find_model(model)
hk.set_seed(si_config)

si_config['n_labels'] = 62
manifest_dir = "manifests/reddots/"
for tag in ['train', 'val', 'test']:
    si_config['{}_manifest'.format(tag)]=os.path.join(manifest_dir,'si_{}_{}_manifest.csv'.format("reddots", tag))

si_model = si_config['model_class'](si_config)
si_config['input_file'] = ""
si_config['output_file'] = "models/reddots/si_reddots_uttrs_res8_moresv.pt"
si_config['input_length'] = int(16000 * 4)
print(si_model)

SpeechResModel (
  (conv0): Conv2d(1, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (pool): AvgPool2d (size=[4, 3], stride=[4, 3], padding=0, ceil_mode=False, count_include_pad=True)
  (bn1): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv1): Conv2d(45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv2): Conv2d(45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn3): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv3): Conv2d(45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn4): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv4): Conv2d(45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn5): BatchNorm2d(45, eps=1e-05, momentum=0.1, affine=False)
  (conv5): Conv2d(45, 45, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn6): BatchNorm2d(45, eps=

In [60]:
hk.train(si_config, model=si_model)

epoch #0, final dev accuracy: 0.1839285714285714
saving best model...
train step #99 accuracy: 0.203125, loss: 3.2450408935546875
epoch #1, final dev accuracy: 0.26584821428571426
saving best model...
train step #199 accuracy: 0.296875, loss: 2.7089650630950928
epoch #2, final dev accuracy: 0.3607142857142857
saving best model...
train step #299 accuracy: 0.40625, loss: 2.0809857845306396
epoch #3, final dev accuracy: 0.42790178571428567
saving best model...
epoch #4, final dev accuracy: 0.5377232142857143
saving best model...
train step #399 accuracy: 0.609375, loss: 1.6424047946929932
epoch #5, final dev accuracy: 0.5756696428571428
saving best model...
train step #499 accuracy: 0.671875, loss: 1.670717716217041
epoch #6, final dev accuracy: 0.6013392857142856
saving best model...
train step #599 accuracy: 0.75, loss: 1.34412682056427
epoch #7, final dev accuracy: 0.6383928571428571
saving best model...
epoch #8, final dev accuracy: 0.5705357142857144
train step #699 accuracy: 0.7968

#### SV Enrollment & Test

In [9]:
import random
random.shuffle(sv_spks)
enroll_spks = sv_spks[:5]
test_spks = sv_spks[5:]

In [10]:
enroll_spks

['f0003', 'm0056', 'm0030', 'm0059', 'm0045']

In [11]:
test_spks

['m0063', 'm0035', 'f0015']

In [12]:
enroll_uttrs = pd.DataFrame()
dev_uttrs = pd.DataFrame()
enroll_pts = 0.3

sv_df = data_df[data_df.spk.isin(sv_spks)]
# splits enroll and dev
for spk in enroll_spks:
    spk_df = sv_df[sv_df.spk == spk]
    assert(len(spk_df) != 0)
    enls = spk_df.sample(frac=enroll_pts)
    devs = spk_df.drop(index=enls.index)
    enroll_uttrs = pd.concat([enls, enroll_uttrs])
    dev_uttrs = pd.concat([devs, dev_uttrs])
    
test_uttrs = sv_df[sv_df.spk.isin(test_spks)]

In [82]:
dev_uttrs.to_pickle("./dev_uttrs.pkl")
test_uttrs.to_pickle("./test_uttrs.pkl")
# dev_uttrs = pd.read_pickle("./dev_uttrs.pkl")
# test_uttrs = pd.read_pickle("./test_uttrs.pkl")

In [13]:
import csv
import random

manifest_dir = "./manifests/reddots/enroll/"
# delete existing files
for file in os.listdir(manifest_dir):
    file_path = os.path.join(manifest_dir, file)
    if os.path.isfile(file_path):
        os.unlink(file_path)
        

for spk in enroll_spks:
    samples = []
    save_path = os.path.join(manifest_dir,'enroll_{}_{}_manifest.csv'.format("reddots", spk))
    with open(save_path, 'w') as f:
        for index, row in enroll_uttrs[enroll_uttrs.spk == spk].iterrows():
            file_path = os.path.join(data_dir, row.spk, row.file)
            label = enroll_spks.index(row.spk)
            sample = ','.join([file_path, str(label)])
            samples.append(sample)
        writer = csv.writer(f, delimiter='\n', quoting=csv.QUOTE_NONE)
        writer.writerow(samples)
        print("{} was written".format(save_path))    

./manifests/reddots/enroll/enroll_reddots_f0003_manifest.csv was written
./manifests/reddots/enroll/enroll_reddots_m0056_manifest.csv was written
./manifests/reddots/enroll/enroll_reddots_m0030_manifest.csv was written
./manifests/reddots/enroll/enroll_reddots_m0059_manifest.csv was written
./manifests/reddots/enroll/enroll_reddots_m0045_manifest.csv was written


In [44]:
si_model = si_config['model_class'](si_config)
si_model.load("models/reddots/si_reddots_uttrs_res8_moresv.pt")

In [45]:
# enrolling speakers
# generating speaker models
si_config['batch_size'] = 1
manifest_dir = "./manifests/reddots/enroll/"
spk_models = dict()
for spk in enroll_spks:
    manifest_path = os.path.join(manifest_dir, 'enroll_{}_{}_manifest.csv'.format("reddots", spk))
    si_config['test_manifest'] =  manifest_path
    spk_models[spk] = hk.enroll_uttr(si_config, model=si_model)

In [26]:
from scipy.spatial.distance import cosine
import itertools

for spk1, spk2 in itertools.combinations(enroll_spks,2):
    score = 1-cosine(spk_models[spk1], spk_models[spk2])
    print("{}, {}: {:.2f}".format(spk1, spk2, score))

f0003, m0056: nan
f0003, m0030: nan
f0003, m0059: nan
f0003, m0045: nan
m0056, m0030: nan
m0056, m0059: nan
m0056, m0045: nan
m0030, m0059: nan
m0030, m0045: nan
m0059, m0045: nan


In [87]:
def pairwise_cosine(spk_model, test_in):
    nb_enroll_spks = len(spk_models.keys())
    scores = np.zeros(nb_enroll_spks)
    for i in range(nb_enroll_spks):
        signature = spk_model[i]
        scores[i] = 1-cosine(test_in, signature)
    return scores

In [92]:
corrects = 0
for idx, row in dev_uttrs.iterrows():
    audio_path = os.path.join(data_dir, row.spk, row.file)
    emb = hk.embed(si_config, si_model, audio_path)
    pred_spk = np.argmax(pairwise_cosine(spk_model, emb))
    label = enroll_spks.index(row.spk)
    if pred_spk == label:
        corrects += 1

print("SI Acc: {}".format(corrects/ len(dev_uttrs)))

SI Acc: 0.9496402877697842


In [93]:
nb_total_uttrs = len(dev_uttrs)+len(test_uttrs)
n_classes = len(enroll_spks)
score_vector = np.zeros((nb_total_uttrs, n_classes))
label_vector = np.zeros((nb_total_uttrs, n_classes))

spk_model = [v for k,v in spk_models.items()]
spk_labels = list(spk_models.keys())

i = 0
for idx, row in dev_uttrs.iterrows():
    audio_path = os.path.join(data_dir, row.spk, row.file)
    emb = hk.embed(si_config, si_model, audio_path)
    score_vector[i, :] = pairwise_cosine(spk_model, emb)
    label = spk_labels.index(row.spk)
    label_vector[i, label] = 1
    i+=1

for idx, row in test_uttrs.iterrows():
    audio_path = os.path.join(data_dir, row.spk, row.file)
    emb = hk.embed(si_config, si_model, audio_path)
    score_vector[i, :] = pairwise_cosine(spk_model, emb)
    i+=1

In [ ]:
from sklearn.metrics import roc_curve, auc
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
eer = dict()
thres = dict()
for i in range(n_classes):
    fpr[i], tpr[i], thres[i] = roc_curve(label_vector[:, i], score_vector[:, i], pos_label=1)
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], thres["micro"] = roc_curve(label_vector.ravel(), score_vector.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])
eer["micro"] = fpr["micro"][np.nanargmin(np.abs(fpr["micro"] - (1 - tpr["micro"])))]

for i in range(n_classes):
    eer[i] = fpr[i][np.nanargmin(np.abs(fpr[i] - (1 - tpr[i])))]

from scipy import interp
from itertools import cycle
lw = 2
# Compute macro-average ROC curve and ROC area
# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])
eer["macro"] = fpr["macro"][np.nanargmin(np.abs(fpr["macro"] - (1 - tpr["macro"])))]
# Plot all ROC curves
fig = plt.figure(figsize=(10,7))
plt.plot(fpr["micro"], tpr["micro"],
         label='micro-average ROC curve (area = {0:0.2f}, eer = {1:0.4f})'
               ''.format(roc_auc["micro"], eer["micro"]),
         color='deeppink', linestyle=':', linewidth=4)

plt.plot(fpr["macro"], tpr["macro"],
         label='macro-average ROC curve (area = {0:0.2f}, eer = {1:0.4f})'
               ''.format(roc_auc["macro"], eer["macro"]),
         color='navy', linestyle=':', linewidth=4)

colors = cycle(['aqua', 'darkorange', 'cornflowerblue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='ROC curve of {0} (area = {1:0.2f}, eer = {2:0.4f})'
             ''.format(spk_labels[i], roc_auc[i], eer[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Some extension of Receiver operating characteristic to multi-class')
plt.legend(loc="lower right")
plt.show()

In [54]:
len(dev_uttrs)

35

Parameter Counts

In [12]:
nb_params = 0
for param in si_model.parameters():
    nb_params += np.prod(param.size())
print(nb_params)    

112607
